In [ ]:
import numpy as np
from enum import Enum

class LearningRule(Enum):
    Hebb = 1
    Oja = 2,

class SimulationType(Enum):
    Sync = 1
    Async = 2,

class HopfieldStructure:
    
    def __init__(self, neurons) -> None:
        self.neurons = neurons
        self.initializeWeights()        
        
        pass
    
    def initializeWeights(self, neurons):
        self.weights = np.array(neurons, neurons)
        
        pass
    
class HopfieldNetwork:
    
    def __init__(self, structure) -> None:
        self.structure = structure
        
        pass
    